In [1]:
!pip install pymupdf pytesseract Pillow


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import fitz  #PyMuPDF
import pytesseract
from PIL import Image
import os
import io
import csv

pdf_dir = "pdf_files_sdg16"  #change to pdf directory
csv_file_path = "DATASET_AP_FINALdipasure - SDG 16.csv"   #change to where the extracted text will be saved

extracted_texts = []

#loop through all the pdfs in the directory
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        file_path = os.path.join(pdf_dir, pdf_file)

        #open pdf
        pdf_document = fitz.open(file_path)

        #placeholder for all the extracted text (images and direct text)
        all_text = ""

        #loop through each page
        for page_number in range(len(pdf_document)):
            page = pdf_document[page_number]

            #extract all visible images in the pdf
            page_text = page.get_text()
            all_text += page_text

            #get all images from the pdf
            images = page.get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                #load image using PIL
                image = Image.open(io.BytesIO(image_bytes))

                #perform OCR on the image
                ocr_text = pytesseract.image_to_string(image)
                all_text += ocr_text

        extracted_texts.append(all_text)

#update the rows in the csv file
updated_rows = []
with open(csv_file_path, mode="r", newline="", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader) 
    for row in reader:
        #fill in all the rows with "" or null values
        if len(row) > 1 and row[1].strip() in ("", "<null>"):
            if extracted_texts:
                row[1] = extracted_texts.pop(0)
                row[headers.index("LABEL/S (SDG)")] = "16"  # Set 'label' column to '16'

        updated_rows.append(row)

#write the updated rows in the csv file
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)  # Write the header row
    writer.writerows(updated_rows)

print(f"Extracted text updated in {csv_file_path}.")


Extracted text updated in DATASET_AP_FINALdipasure - SDG 16.csv.
